In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import requests
import getpass
import pandas as pd

from bs4 import BeautifulSoup
from IPython.display import display

In [46]:
class ConfluenceUtils(object):
    """Confluence API 연동 클래스"""

    def __init__(self):
        """ """
        self.auth = ()

        page_info = '''
56302671	크롤링_사전
56302696	크롤링_L2
56302702	크롤링_L2M
56302703	크롤링_BNS
56302706	크롤링_문어/크롤링_뉴스
56302708	크롤링_구어/크롤링_기타 게임
56302711	크롤링_구어/크롤링_자막
'''
        self.page_list = {}
        
        for line in page_info.split('\n'):
            line = line.strip()
            
            if line == '':
                continue
                
            (doc_id, title) = line.split('\t', maxsplit=1)
            self.page_list[title] = doc_id

        self.api_url = 'http://172.20.93.58:8090/rest/api/content/{page_id}?expand=space,body.view,version,container'

    def get_page(self, page_id):
        """ 문서 내용 조회 """
        url = self.api_url.format(page_id=page_id)

        resp = requests.get(url=url, auth=self.auth, timeout=30, verify=False)
        doc = resp.json()

        return doc['body']['view']['value']
    
    def batch(self):
        """ """
        user_id = input('user id:')
        password = getpass.getpass('password:')
        
        self.auth = (user_id, password)
        
        result = []
        for title in self.page_list.keys():
            print(title, self.page_list[title])
            html = self.get_page(page_id=self.page_list[title])
            
            tbl_list = [tbl.fillna('') for tbl in pd.read_html(html, header=0)]
            result += tbl_list

            for x in result:
                x['category'] = title
            
        return result
    
utils = ConfluenceUtils()

df_list = utils.batch()

user id:ejpark
password:········
크롤링_사전 56302671
크롤링_L2 56302696
크롤링_L2M 56302702
크롤링_BNS 56302703
크롤링_문어/크롤링_뉴스 56302706
크롤링_구어/크롤링_기타 게임 56302708
크롤링_구어/크롤링_자막 56302711


In [9]:
for df in df_list:
    display(df)

,언어,유형,명칭,URL,비고,진행 상황,category
0,KO-VI,사전,Vdict 사전,http://vdict.co/index.php?word=&dict=kr_vi,한-베 검색 시 해당 베어 없는 경우 한-영으로 자동전환,,크롤링_구어/크롤링_자막
1,KO-VI,사전,Co Viet 사전,http://tratu.coviet.vn/tu-dien-lac-viet.aspx?l...,예문 제공,,크롤링_구어/크롤링_자막
2,KO-VI,사전,Lingega 사전,https://www.dict.com/han-viet/,베-한 미제공,,크롤링_구어/크롤링_자막
3,KO-VI,사전,Glosbe 사전,https://glosbe.com/ko/vi/,관련 표현 및 예문 제공,개발 완료,크롤링_구어/크롤링_자막
4,KO-VI,사전,V 사전,https://vtudien.com/han-viet,베-한 미제공,,크롤링_구어/크롤링_자막
5,KO-VI,사전,국립국어원 사전,https://krdict.korean.go.kr/vie/mainAction?nat...,예문 제공,,크롤링_구어/크롤링_자막


,언어,유형,명칭,URL,비고,진행 상황,category
0,VI-EN,사전,Tra cau 사전,https://tracau.vn/,베-영 예문 제공,,크롤링_구어/크롤링_자막
1,VI-EN,사전,Tra tu 사전,http://tratu.soha.vn/,예문 미제공하나 관련 표현 다양하게 제공,,크롤링_구어/크롤링_자막
2,VI-EN,사전,Babla 사전,https://www.babla.vn/tieng-viet-tieng-anh/,예문 제공,,크롤링_구어/크롤링_자막
3,VI-EN,사전,V 사전,https://vtudien.com/,예문 미제공하나 관련 표현 다양하게 제공,,크롤링_구어/크롤링_자막


,언어,유형,명칭,URL,비고,진행 상황,category
0,EN-VI,사전,Cambridge 사전,https://dictionary.cambridge.org/vi/dictionary...,"베-영 검색 불가, 영-베 검색 시 영어 예문 제공",,크롤링_구어/크롤링_자막
1,EN-VI,사전,VN dict 사전,http://5.vndic.net/,영-베 예문 제공,,크롤링_구어/크롤링_자막
2,EN-VI,사전,Tra tu 사전,http://tratu.coviet.vn/,관련 표현 제공,,크롤링_구어/크롤링_자막
3,EN-VI,사전,La ban 사전,https://dict.laban.vn/,영-베 예문 제공 (한번 검색 시 한 문장씩 예문 제공),,크롤링_구어/크롤링_자막
4,EN-VI,사전,Rung 사전,https://www.rung.vn/,영-베 관련 표현 제공,,크롤링_구어/크롤링_자막
5,EN-VI,사전,Dict 사전,https://www.dict.com/anh-viet,예문 미제공,,크롤링_구어/크롤링_자막


,언어,유형,명칭,URL,비고,진행 상황,category
0,KO-ID,사전,krdict 사전,https://krdict.korean.go.kr/m/ind/help?nation=ind,한-인니어로 검색 시 정의 및 예문 제공,,크롤링_구어/크롤링_자막
1,KO-ID,사전,Glosbe 사전,https://id.glosbe.com/ko/id,인니어-한국어 검색시 관련 표현 및 예문 제공,,크롤링_구어/크롤링_자막
2,KO-ID,사전,17minutelanguages 사전,https://www.17-minute-languages.com/id/kamus-b...,한-인니어로 검색시 예문 제공함,,크롤링_구어/크롤링_자막
3,KO-ID,사전,dict.com 사전,https://www.dict.com/korea-indonesia,한-인니어로 검색시 예문 및 어휘 관련 표현 제공함,,크롤링_구어/크롤링_자막
4,ID-KO,사전,17minutelanguage 사전,https://www.17-minute-languages.com/id/kamus-b...,인니어-한국어로 검색시 어휘 관련 표현 제공,,크롤링_구어/크롤링_자막
5,ID-KO,사전,한국어 기초 사전,https://krdict.korean.go.kr/mainAction,한국어-여러나라 언어 사전 (vi/id/arabic/eng/spn 등),,크롤링_구어/크롤링_자막


,언어,유형,명칭,URL,비고,진행 상황,category
0,EN-CN,사전,,https://dict.naver.com/linedict/enzhdict/#/enc...,크롤링 진행crawler-dictionary-example-linedict,크롤링 완료,크롤링_구어/크롤링_자막
1,CN-EN,사전,,http://dict.eudic.net/,(CN) EU 사전 구조크롤링 진행crawler-dictionary-example-...,크롤링 완료,크롤링_구어/크롤링_자막
2,CN-EN,사전,,http://www.youdao.com/,(CN) 유도 사전 구조,크롤링 완료,크롤링_구어/크롤링_자막
3,CN-EN,사전,유도 사전,http://www.youdao.com/,예문 많음,크롤링 완료,크롤링_구어/크롤링_자막
4,CN-EN,사전,콜린스 사전,https://www.collinsdictionary.com/zh/dictionar...,,,크롤링_구어/크롤링_자막
5,CN-EN,사전,해사 사전,http://dict.cn/,단어당 예문 8개씩,,크롤링_구어/크롤링_자막
6,CN-EN,사전,킹소프트 사전,http://www.iciba.com/,예문 더 보기 → 403 forbidden,,크롤링_구어/크롤링_자막
7,CN-EN,사전,BING 사전,https://www.bing.com/dict?FORM=Z9LH3,,,크롤링_구어/크롤링_자막
8,CN-EN,사전,EU 사전,http://dict.eudic.net/?aspxerrorpath=/dicts/en/,예문 메뉴가 따로 존재,크롤링 완료,크롤링_구어/크롤링_자막


...

,언어,유형,명칭,주소,설명,진행 상황,category
0,KO,뉴스,베트남 정보 비나타임즈,https://www.vinatimes.net/vn/,,,크롤링_구어/크롤링_자막
1,KO,뉴스,베한 타임즈,http://www.viethantimes.com/,,,크롤링_구어/크롤링_자막
2,KO,뉴스,비나 한인,https://www.vinahanin.com/vietnam_news,,,크롤링_구어/크롤링_자막
3,KO,뉴스,인사이드 비나,http://www.insidevina.com/,,,크롤링_구어/크롤링_자막
4,KO,뉴스,신짜오 베트남,http://www.chaovietnam.co.kr,,,크롤링_구어/크롤링_자막
5,KO,뉴스,베트남 투데이,http://www.vietnam2day.com/,,,크롤링_구어/크롤링_자막
6,KO,뉴스,베트남 라이프,https://vietnamlife.co.kr/,,,크롤링_구어/크롤링_자막
7,KO,뉴스,한베 비즈니스 캠프,http://www.hanbe.kr/,,,크롤링_구어/크롤링_자막
8,KO,포털,하노이 114,http://www.hanoi114.net/,하노이 생활 정보,,크롤링_구어/크롤링_자막
9,KO,포털,베트남 갑시다,http://vietnamgo.co.kr/index.php3,베트남 여행 정보,,크롤링_구어/크롤링_자막


,언어,유형,명칭,주소,설명,진행 상황,category
0,EN,뉴스,Formosa News,https://englishnews.ftv.com.tw/,일반 뉴스,,크롤링_구어/크롤링_자막
1,EN,뉴스,The China Post,https://chinapost.nownews.com/,일반 뉴스,,크롤링_구어/크롤링_자막
2,EN,뉴스,Liberty Times Net,https://features.ltn.com.tw/english,일반 뉴스,,크롤링_구어/크롤링_자막
3,EN,뉴스,Taiwan News/台灣英文新聞,https://www.taiwannews.com.tw/en/index,일반 뉴스,,크롤링_구어/크롤링_자막
4,EN,뉴스,TAIPEI TIMES/台北時報,https://www.taipeitimes.com/News/taiwan,일반 뉴스,,크롤링_구어/크롤링_자막
5,EN,뉴스,Taiwan's News.Net,https://www.taiwansnews.net/,일반 뉴스,,크롤링_구어/크롤링_자막
6,EN,뉴스,Taiwan.gov.tw,https://www.taiwan.gov.tw/news.php,일반 뉴스,,크롤링_구어/크롤링_자막
7,EN,뉴스,Taiwans English News,http://taiwanenglishnews.com/,일반 뉴스,,크롤링_구어/크롤링_자막
8,EN,뉴스,The News Lens,https://international.thenewslens.com/,일반 뉴스,,크롤링_구어/크롤링_자막
9,EN,뉴스,ICRT Taiwan News,https://www.icrt.com.tw/info_list01.php?&mleve...,일반 뉴스,,크롤링_구어/크롤링_자막


,언어,유형,명칭,주소,설명,진행 상황,category
0,중국어 간체,페이스북-기타 게임,英雄联盟,https://www.facebook.com/LeagueofLegendsChina/,,크롤링 중,크롤링_구어/크롤링_자막
1,중국어 간체,페이스북-기타 게임,绝地求生 Pubg,https://www.facebook.com/%E7%BB%9D%E5%9C%B0%E6...,,크롤링 중,크롤링_구어/크롤링_자막
2,중국어 간체,페이스북-기타 게임,绝地求生大逃杀 Playerunknown's Battlegrounds,https://www.facebook.com/jdqspubg/,,크롤링 중,크롤링_구어/크롤링_자막
3,중국어 간체,페이스북-기타 게임,绝地求生Asia,https://www.facebook.com/PubgAsia2/,,크롤링 중,크롤링_구어/크롤링_자막
4,중국어 간체,페이스북-기타 게임,王者荣耀 最强攻略,https://www.facebook.com/wzryzqgl/,,크롤링 중,크롤링_구어/크롤링_자막
5,중국어 간체,페이스북-기타 게임,王者荣耀Asia,https://www.facebook.com/gloryofthekingasia/,,크롤링 중,크롤링_구어/크롤링_자막
6,중국어 간체,페이스북-기타 게임,王者荣耀-互动游戏,https://www.facebook.com/WZRY1116/,,크롤링 중,크롤링_구어/크롤링_자막
7,중국어 간체,페이스북-기타 게임,一起游戏,https://www.facebook.com/17UC8/,,크롤링 중,크롤링_구어/크롤링_자막
8,중국어 간체,페이스북-기타 게임,王者有毒,https://www.facebook.com/WangZheYouDu/,,크롤링 중,크롤링_구어/크롤링_자막
9,중국어 간체,페이스북-기타 게임,王者巅峰大神,https://www.facebook.com/wzdfds/,,크롤링 중,크롤링_구어/크롤링_자막


,언어,유형,명칭,주소,설명,진행 상황,category
0,중국어 번체,페이스북-기타 게임,英雄聯盟,https://www.facebook.com/LeagueOfLegendsTaiwan/,,크롤링 중,크롤링_구어/크롤링_자막
1,중국어 번체,페이스북-기타 게임,英雄聯盟 - LoL精彩影片,https://www.facebook.com/GameMadman/,,크롤링 중,크롤링_구어/크롤링_자막
2,중국어 번체,페이스북-기타 게임,英雄联盟手游 LOL Mobile,https://www.facebook.com/%E8%8B%B1%E9%9B%84%E8...,,크롤링 중,크롤링_구어/크롤링_자막
3,중국어 번체,페이스북-기타 게임,英雄聯盟精彩影音,https://www.facebook.com/LovetheLeagueoflegends/,,크롤링 중,크롤링_구어/크롤링_자막


,Unnamed: 0,간체/번체,페이지 명,링크,소개,비고,진행 상황,category
0,1,번체+간체,자막 다운로드,http://www.zmxiazai.com/,영화/드라마 자막,열리고 다운은 안됨,,크롤링_구어/크롤링_자막
1,2,번체+간체,자막 창고,http://www.zimuku.la/newsubs?t=tv&ad=1,영화/드라마 자막,열리고 다운도 됨,,크롤링_구어/크롤링_자막
2,3,번체+간체,차막 천국,http://www.zmtiantang.cc/,영화/드라마 자막,열리고 다운은 안됨,,크롤링_구어/크롤링_자막
3,4,간체(추정),알 수 없음,http://www.opensubtitles.org/zh,영화/드라마 자막,회사에서 안 열림,,크롤링_구어/크롤링_자막
4,5,번체,大紀元寓言故事中英文版,https://www.epochtimes.com/b5/nf2875.htm,우화,,,크롤링_구어/크롤링_자막
5,6,번체+간체,Sub HD,subhd.com,영화/드라마 자막,회사에서 열리나 다운로드 안 됨,,크롤링_구어/크롤링_자막
6,7,간체,문서 창고,http://www.wendangku.net/search/%e5%90%ac%e5%8...,듣기 시험 스크립트,,,크롤링_구어/크롤링_자막
7,8,번체,中英對照讀新聞,https://features.ltn.com.tw/english/,뉴스,,,크롤링_구어/크롤링_자막
8,9,간체,,https://wenku.baidu.com/view/fdc85686182e45361...,듣기 시험 스크립트,,,크롤링_구어/크롤링_자막
9,10,번체+간체,유튜브,"(필요 시 검색 후 적절한 것으로 더 추가 예정)한국 웹드라마 ""소울플레이트"" : ...","키워드 EN/TW, EN/CN",,,크롤링_구어/크롤링_자막


In [62]:
df = pd.concat(df_list).fillna('')

df

,언어,유형,명칭,URL,비고,진행 상황,category,예문(boy) 입력 형태,이름,Unnamed: 3,...,다국어서비스제공여부,기준 언어,서비스 언어,동일 기사의 다국어본 제공 여부,기사별 링크 제공 여부,진행상황,간체/번체,페이지 명,링크,소개
0,KO-VI,사전,Vdict 사전,http://vdict.co/index.php?word=&dict=kr_vi,한-베 검색 시 해당 베어 없는 경우 한-영으로 자동전환,,크롤링_구어/크롤링_자막,,,,...,,,,,,,,,,
1,KO-VI,사전,Co Viet 사전,http://tratu.coviet.vn/tu-dien-lac-viet.aspx?l...,예문 제공,,크롤링_구어/크롤링_자막,,,,...,,,,,,,,,,
2,KO-VI,사전,Lingega 사전,https://www.dict.com/han-viet/,베-한 미제공,,크롤링_구어/크롤링_자막,,,,...,,,,,,,,,,
3,KO-VI,사전,Glosbe 사전,https://glosbe.com/ko/vi/,관련 표현 및 예문 제공,개발 완료,크롤링_구어/크롤링_자막,,,,...,,,,,,,,,,
4,KO-VI,사전,V 사전,https://vtudien.com/han-viet,베-한 미제공,,크롤링_구어/크롤링_자막,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,,,,,,,크롤링_구어/크롤링_자막,,,,...,,,,,,,간체,문서 창고,http://www.wendangku.net/search/%e5%90%ac%e5%8...,듣기 시험 스크립트
7,,,,,,,크롤링_구어/크롤링_자막,,,,...,,,,,,,번체,中英對照讀新聞,https://features.ltn.com.tw/english/,뉴스
8,,,,,,,크롤링_구어/크롤링_자막,,,,...,,,,,,,간체,,https://wenku.baidu.com/view/fdc85686182e45361...,듣기 시험 스크립트
9,,,,,,,크롤링_구어/크롤링_자막,,,,...,,,,,,,번체+간체,유튜브,"(필요 시 검색 후 적절한 것으로 더 추가 예정)한국 웹드라마 ""소울플레이트"" : ...","키워드 EN/TW, EN/CN"


In [77]:
url_list = list(df[(df['주소'] != '') & (df['진행 상황'] == '') & (df['주소'].str.find('http') == 0)]['주소'])
url_list, len(url_list)

(['https://forum.gamer.com.tw/B.php?bsn=05264',
  'https://discordapp.com/invite/l2classiceu',
  'https://discordapp.com/invite/DrpeY9c',
  'https://discordapp.com/invite/cJ562eE',
  'https://discordapp.com/invite/5QqztUR',
  'https://forums.hopzone.net/forum/pc-gaming/lineage-2',
  'https://l2topzone.com/forum/index.php',
  'https://steamcommunity.com/app/373700/discussions/',
  'https://tiantang2.qq.com/',
  'https://twitter.com/lineage2_fs',
  'https://twitter.com/Line2Revo',
  'https://twitter.com/LineageM_JP',
  'https://twitter.com/line2Cwiki',
  'https://www.ncsoft.jp/community/latest/list?category=2',
  'https://www.ncsoft.jp/lineage2classic/community/lineage2Board/list',
  'https://www.ncsoft.jp/lineage2aden/community/lineage2Board/list',
  'https://tiantang2.gamebbs.qq.com/forum.php',
  'http://www.l2cn.com/',
  'http://tt2.bbs.woniu.com/',
  'http://lineage2.17173.com/',
  'http://qq.sgtt2.com/',
  'http://www.168gamer.com/forum-77-1.html',
  'http://www.gamez.com.tw/forum-7

In [88]:
skip_list = 'facebook,youtube,qq,youku,cafe.naver,ncsoft,kakao,daum,twitter,.qq,kakao'.split(',')

result = []

for url in url_list:
    skip = False
        
    for k in skip_list:
        if k in url:
            skip = True
            break
            
    if skip is True:
        continue
        
    result.append(url)

sorted(result), len(result)

(['http://bali-travelnews.com/',
  'http://bbs.17173.com/forum-2468-2.html',
  'http://bbs.l2sd.com/',
  'http://bns.duowan.com/tag/156856402111.html',
  'http://bns.gamersky.com/',
  'http://bns.hehagame.com/ShowArt155053.html',
  'http://bns.inven.co.kr/',
  'http://bns.pcgames.com.cn/',
  'http://bns.plaync.com/board/free/article',
  'http://bns.tgbus.com/',
  'http://dailyindonesia.co.kr/',
  'http://forum.gamevn.com/forums/lineage-2.217/',
  'http://forum.l2r.vn/lin2vn/view/4/30',
  'http://forum.netmarble.com/lineage2_id/',
  'http://gamehub.vn/hub/lineage-2-revolution-vietnam-chinh-thuc-cong-bo-su-xuat-hien-cua-toc-moi-kamael.137748/',
  'http://gametsg.techbang.com/lineage2/',
  'http://gametsg.techbang.com/lineage2/',
  'http://haninpost.com/',
  'http://hanoitimes.vn/',
  'http://l2vn.com/forum/',
  'http://lineage2.17173.com/',
  'http://lineage2.inven.co.kr/',
  'http://logu.jp/',
  'http://news.kbs.co.kr/news/view.do?ncd=4465295',
  'http://news.sina.com.cn/area/tw.shtml',

In [89]:
with open('url.list', 'w') as fp:
    fp.write('\n'.join(sorted(result)))

In [87]:
!head url.list

http://bali-travelnews.com/
http://bbs.17173.com/forum-2468-2.html
http://bbs.l2sd.com/
http://bns.duowan.com/tag/156856402111.html
http://bns.gamersky.com/
http://bns.hehagame.com/ShowArt155053.html
http://bns.inven.co.kr/
http://bns.pcgames.com.cn/
http://bns.plaync.com/board/free/article
http://bns.tgbus.com/


https://unix.stackexchange.com/questions/84015/how-to-download-all-html-files-from-a-urls-directory-with-wget
https://superuser.com/questions/709702/how-to-crawl-using-wget-to-download-only-html-files-ignore-images-css-js

```
wget -r -np -k -p -e robots=off --no-parent -l 100 --wait=5 -q --show-progress --progress=bar:force --directory-prefix=down $URL
```

> The args (see man wget) are:

> r Recurse into links, retrieving those pages too (this has a default max depth of 5, can be set with -l).
np Never enter a parent directory (i.e., don't follow a "home" link and mirror the whole site; this will prevent going above ccc in your example).

> k Convert links relative to local copy.

> p Get page-requisites like stylesheets (this is an exception to the np rule).

> If I remember correctly, wget will create a directory name


In [84]:
!wget \
    -r -np -k -p --no-netrc \
    -e robots=off --no-parent \
    --wait=5 -l 100 \
    -q --show-progress --progress=bar:force \
    --directory-prefix=down \
    --reject '*.js,*.css,*.ico,*.txt,*.gif,*.jpg,*.jpeg,*.png,*.mp3,*.pdf,*.tgz,*.flv,*.avi,*.mpeg,*.iso' \
    --ignore-tags=img,link,script \
    --header="Accept: text/html" \
    http://bns.inven.co.kr/

wget: /home/ejpark/.netrc:2: unknown token "protocol"
wget: /home/ejpark/.netrc:2: unknown token "https"
wget: /home/ejpark/.netrc:7: unknown token "protocol"
wget: /home/ejpark/.netrc:7: unknown token "https"
bns.inven.co.kr/ind     [ <=>                ] 355.69K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 200.26K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 241.61K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 716.18K  --.-KB/s    in 0.08s   
bns.inven.co.kr/dat     [ <=>                ] 367.99K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 198.90K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 194.33K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.76K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 310.53K  --.-KB/s    in 0.03s   
bns.inven.co.kr/com     [ <=>                ] 223.32K  --.-KB/s    in

bns.inven.co.kr/dat     [ <=>                ] 441.08K  --.-KB/s    in 0.05s   
bns.inven.co.kr/dat     [ <=>                ] 440.84K  --.-KB/s    in 0.05s   
bns.inven.co.kr/dat     [ <=>                ] 440.93K  --.-KB/s    in 0.05s   
bns.inven.co.kr/dat     [ <=>                ] 441.06K  --.-KB/s    in 0.04s   
bns.inven.co.kr/dat     [ <=>                ] 582.66K  --.-KB/s    in 0.06s   
bns.inven.co.kr/dat     [ <=>                ] 230.92K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 456.97K  --.-KB/s    in 0.06s   
bns.inven.co.kr/dat     [ <=>                ] 231.07K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 467.36K  --.-KB/s    in 0.04s   
bns.inven.co.kr/dat     [ <=>                ] 461.80K  --.-KB/s    in 0.04s   
bns.inven.co.kr/dat     [ <=>                ] 217.03K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 213.47K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

bns.inven.co.kr/dat     [ <=>                ] 215.64K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 194.95K  --.-KB/s    in 0.04s   
bns.inven.co.kr/dat     [ <=>                ] 194.96K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.53K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.60K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 215.14K  --.-KB/s    in 0.04s   
bns.inven.co.kr/dat     [ <=>                ] 215.24K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 207.21K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 207.29K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.42K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.49K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.27K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

bns.inven.co.kr/dat     [ <=>                ] 196.23K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 196.29K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 196.36K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 196.40K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 196.04K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 196.07K  --.-KB/s    in 0.05s   
bns.inven.co.kr/dat     [ <=>                ] 195.07K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 195.12K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 194.88K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 194.92K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 209.26K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 209.36K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

bns.inven.co.kr/dat     [ <=>                ] 195.11K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.01K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.03K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 205.97K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.01K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.23K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.28K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.74K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.83K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 199.71K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 199.72K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 200.74K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

...

bns.inven.co.kr/dat     [ <=>                ] 208.22K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 207.93K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.12K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.00K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.41K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.40K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.42K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.43K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.74K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.75K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.71K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 208.74K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

bns.inven.co.kr/dat     [ <=>                ] 217.27K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.23K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.25K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 217.28K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.25K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.24K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.25K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.29K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.26K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.25K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.27K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.25K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

bns.inven.co.kr/dat     [ <=>                ] 217.92K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.92K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 217.92K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.95K  --.-KB/s    in 0.04s   
bns.inven.co.kr/dat     [ <=>                ] 217.93K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.91K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.96K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.92K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.94K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 217.95K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.90K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 217.91K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

bns.inven.co.kr/dat     [ <=>                ] 202.39K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.40K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.39K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.38K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.38K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.41K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.39K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.42K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 202.39K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 211.08K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 211.44K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 211.30K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

bns.inven.co.kr/dat     [ <=>                ] 206.05K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.07K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.03K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.04K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.04K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.07K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 206.02K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.06K  --.-KB/s    in 0.03s   
bns.inven.co.kr/dat     [ <=>                ] 206.06K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.08K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.02K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>                ] 206.04K  --.-KB/s    in 0.02s   
bns.inven.co.kr/dat     [ <=>           

In [90]:
cmd = '''wget \
-r -np -k -p --no-netrc \
-e robots=off --no-parent \
--wait=5 -l 100 \
-q --show-progress --progress=bar:force \
--directory-prefix=down \
--reject '*.js,*.css,*.ico,*.txt,*.gif,*.jpg,*.jpeg,*.png,*.mp3,*.pdf,*.tgz,*.flv,*.avi,*.mpeg,*.iso' \
--ignore-tags=img,link,script \
--header="Accept: text/html" "{}"'''

In [93]:
cmd_list = []
for url in result:
    cmd_list.append(cmd.format(url))
    
with open('cmd.list', 'w') as fp:
    fp.write('\n'.join(cmd_list))